In [6]:
#pip install pandas langchain openai tiktoken

from langchain_core.documents import Document

In [11]:
import pandas as pd
from pathlib import Path

# 1. 입력 CSV 경로들
CSV_FILES = [
    "data/processed/law/law.csv",
    "data/processed/rule/rule.csv",
    "data/processed/case/case.csv",
]

# 2. CSV 로드
dfs = []
for path in CSV_FILES:
    df = pd.read_csv(path)
    dfs.append(df)

# 3. 하나로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 4. (선택) id 중복 체크
dup_ids = merged_df["id"].duplicated().sum()
print(f"중복 id 개수: {dup_ids}")

# 5. 저장
output_path = Path("data/processed/all_chunks.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

merged_df.to_csv(output_path, index=False)

print(f"✅ 병합 완료: {output_path}")
print(f"총 청크 수: {len(merged_df)}")

중복 id 개수: 0
✅ 병합 완료: data\processed\all_chunks.csv
총 청크 수: 248


In [14]:
import pandas as pd
from langchain_core.documents import Document

def load_chunked_csv(csv_path: str):
    df = pd.read_csv(csv_path)

    documents = []
    for _, row in df.iterrows():
        documents.append(
            Document(
                page_content=row["text"],
                metadata={
                    "id": row["id"],
                    "category": row["category"],
                    "priority": int(row["priority"]),
                    "article": row["article"],
                    "chunk_id": row["chunk_id"],
                    "source": row["source"],
                }
            )
        )
    return documents

In [15]:
docs = load_chunked_csv("data/processed/all_chunks.csv")

In [17]:
docs[0]

Document(metadata={'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 1, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}, page_content='(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차')

In [18]:
docs[0].page_content

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [19]:
docs[0].page_content[:300]

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [20]:
for i in range(5):
    print(f"\n--- doc {i} ---")
    print(docs[i].metadata)
    print(docs[i].page_content[:200])


--- doc 0 ---
{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 1, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.
제2장 계약
제6절 사용대차

--- doc 1 ---
{'id': '767cb115-6be6-441b-a624-0de4fdd5d05d', 'category': 'law', 'priority': 1, 'article': '제618조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에 대하여 차임을 지급할 것을 약정함으로써 그 효력이 생긴다.

--- doc 2 ---
{'id': '1bb029e6-f430-4d8e-8777-f93e7ac57c90', 'category': 'law', 'priority': 1, 'article': '제619조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(처분능력, 권한없는 자의 할 수 있는 단기임대차) 처분의 능력 또는 권한없는 자가 임대차를 하는 경우에는 그 임대차는 다음 각호의 기간을 넘지 못한다.
1. 식목, 채염 또는 석조, 석회조, 연와조 및 이와 유사한 건축을 목적으로 한 토지의 임대차는 10년
2. 기타 토지의 임대차는 5년
3. 건물 기타 공작물의 임대차는 3년
4. 동산의 임대차는 6월

--- doc 3 ---
{'id': '1527dfbd-1069-